# 1. ABSTRACT:

## In this ETL prject, we wanted to obtain a data set would help us answer the question "Do housing prices and other factors fluctuate due to the quality of the schools in the area?".  In this scenario, our hypothesis would be that our dependent, housing prices, do have a notable correlation with the quality of the schools in the are.

## To obtain our data set, we have scrapped realtor.com and usnews.com, respectively, to obtain information on house prices/ charaacteristis in comparrison to school qualities in different areas around St. Louis.  We will explain or Extract, Transform, and Load procedures in further detail below.

##  *****Quick note... To view all other parts of our project, please visit the house scraping and school scraping pages.

In [81]:
# Import dependencies for the load
import pymongo
from pprint import pprint

import json
import pandas as pd

In [82]:
Raw_House_Data = "Scraped_houses.csv"
house_df = pd.read_csv(Raw_House_Data)

school_district_data = "schools_districts_raw.csv"
school_df = pd.read_csv(school_district_data)

# Check DF
house_df.head()

,Unnamed: 0,Property Types,Price,Street Addresses,City,State,Zipcode,Sqft Area,Beds,Baths,Links
0,0,House for Sale,"$279,900","147 Country Creek Ct,",Ballwin,MO,63011,"2,001",3,2.5,https://www.realtor.com//realestateandhomes-de...
1,1,House for Sale,"$350,000","349 Cherry Hill Dr,",Ellisville,MO,63011,"2,397",3,2.5,https://www.realtor.com//realestateandhomes-de...
2,2,House for Sale,"$340,000","402 Baberton Dr,",Ballwin,MO,63011,"2,419",4,2.5,https://www.realtor.com//realestateandhomes-de...
3,3,House for Sale,"$319,900","302 Wildbrier Dr,",Ballwin,MO,63011,"2,319",5,2,https://www.realtor.com//realestateandhomes-de...
4,4,House for Sale,"$199,900","607 Morewood Ct,",Manchester,MO,63011,"1,376",3,2,https://www.realtor.com//realestateandhomes-de...


In [83]:
# Check DF
school_df.head()

,Unnamed: 0,school_name,school_distrct,MO_raniking,national_ranking,city,state,zipcode,full_address,phone_number,school_page_link_usn
0,0,Lafayette Sr. High,Rockwood R-Vi,1,431,Wildwood,MO,63011,"17050 Clayton Rd, Wildwood, Missouri 63011",(636) 733-4100,https://www.usnews.com//education/best-high-sc...
1,1,Smithville High,Smithville R-II,2,531,Smithville,MO,64089,"645 S Commercial Ave, Smithville, Missouri 64089",(816) 532-0405,https://www.usnews.com//education/best-high-sc...
2,2,Rockwood Summit Sr. High,Rockwood R-Vi,3,621,Fenton,MO,63026,"1780 Hawkins Rd, Fenton, Missouri 63026",(636) 891-6800,https://www.usnews.com//education/best-high-sc...
3,3,Eureka Sr. High,Rockwood R-Vi,4,681,Eureka,MO,63025,"4525 Hwy 109, Eureka, Missouri 63025",(636) 733-3100,https://www.usnews.com//education/best-high-sc...
4,4,Metro Academic and Classical High School,St. Louis Public Schools,5,799,St Louis,MO,63108,"4015 Mcpherson Ave, St Louis, Missouri 63108",(314) 534-3894,https://www.usnews.com//education/best-high-sc...


# 2. TRANSFORM - Reduce, Clean, Merge...

## Great, we recived almost 300 auality results for the schools data.  Next we will clean and reshape our data to formulate it in a manner that can help tell a comprehensive story of the future findings.

In [84]:
# Delet the unneeded column
del school_df['Unnamed: 0']
del house_df['Unnamed: 0']

In [85]:
# Check house DF
house_df.head()

,Property Types,Price,Street Addresses,City,State,Zipcode,Sqft Area,Beds,Baths,Links
0,House for Sale,"$279,900","147 Country Creek Ct,",Ballwin,MO,63011,"2,001",3,2.5,https://www.realtor.com//realestateandhomes-de...
1,House for Sale,"$350,000","349 Cherry Hill Dr,",Ellisville,MO,63011,"2,397",3,2.5,https://www.realtor.com//realestateandhomes-de...
2,House for Sale,"$340,000","402 Baberton Dr,",Ballwin,MO,63011,"2,419",4,2.5,https://www.realtor.com//realestateandhomes-de...
3,House for Sale,"$319,900","302 Wildbrier Dr,",Ballwin,MO,63011,"2,319",5,2,https://www.realtor.com//realestateandhomes-de...
4,House for Sale,"$199,900","607 Morewood Ct,",Manchester,MO,63011,"1,376",3,2,https://www.realtor.com//realestateandhomes-de...


In [86]:
# Check school DF
school_df.head()

,school_name,school_distrct,MO_raniking,national_ranking,city,state,zipcode,full_address,phone_number,school_page_link_usn
0,Lafayette Sr. High,Rockwood R-Vi,1,431,Wildwood,MO,63011,"17050 Clayton Rd, Wildwood, Missouri 63011",(636) 733-4100,https://www.usnews.com//education/best-high-sc...
1,Smithville High,Smithville R-II,2,531,Smithville,MO,64089,"645 S Commercial Ave, Smithville, Missouri 64089",(816) 532-0405,https://www.usnews.com//education/best-high-sc...
2,Rockwood Summit Sr. High,Rockwood R-Vi,3,621,Fenton,MO,63026,"1780 Hawkins Rd, Fenton, Missouri 63026",(636) 891-6800,https://www.usnews.com//education/best-high-sc...
3,Eureka Sr. High,Rockwood R-Vi,4,681,Eureka,MO,63025,"4525 Hwy 109, Eureka, Missouri 63025",(636) 733-3100,https://www.usnews.com//education/best-high-sc...
4,Metro Academic and Classical High School,St. Louis Public Schools,5,799,St Louis,MO,63108,"4015 Mcpherson Ave, St Louis, Missouri 63108",(314) 534-3894,https://www.usnews.com//education/best-high-sc...


In [87]:
school_district_df_reduced = school_df[['school_name', 'school_distrct', 'MO_raniking', 'national_ranking', 'zipcode']]

school_district_df_reduced.head()

,school_name,school_distrct,MO_raniking,national_ranking,zipcode
0,Lafayette Sr. High,Rockwood R-Vi,1,431,63011
1,Smithville High,Smithville R-II,2,531,64089
2,Rockwood Summit Sr. High,Rockwood R-Vi,3,621,63026
3,Eureka Sr. High,Rockwood R-Vi,4,681,63025
4,Metro Academic and Classical High School,St. Louis Public Schools,5,799,63108


In [88]:
school_district_df_reduced.count()

school_name         291
school_distrct      291
MO_raniking         291
national_ranking    291
zipcode             291
dtype: int64

In [89]:
house_df["Price"]=[x[1:] for x in house_df["Price"]]
house_df["Price"]=[x.replace(',','') for x in house_df["Price"]]
house_df["Sqft Area"]=[x.replace(',','') for x in house_df["Sqft Area"]]
house_df.head()

,Property Types,Price,Street Addresses,City,State,Zipcode,Sqft Area,Beds,Baths,Links
0,House for Sale,279900,"147 Country Creek Ct,",Ballwin,MO,63011,2001,3,2.5,https://www.realtor.com//realestateandhomes-de...
1,House for Sale,350000,"349 Cherry Hill Dr,",Ellisville,MO,63011,2397,3,2.5,https://www.realtor.com//realestateandhomes-de...
2,House for Sale,340000,"402 Baberton Dr,",Ballwin,MO,63011,2419,4,2.5,https://www.realtor.com//realestateandhomes-de...
3,House for Sale,319900,"302 Wildbrier Dr,",Ballwin,MO,63011,2319,5,2,https://www.realtor.com//realestateandhomes-de...
4,House for Sale,199900,"607 Morewood Ct,",Manchester,MO,63011,1376,3,2,https://www.realtor.com//realestateandhomes-de...


In [90]:
house_df["Price"]=house_df["Price"].astype(float)
house_df["Sqft Area"]=house_df["Sqft Area"].astype(float)

In [91]:

house_df["Price/Sqft"]=round(house_df["Price"]/house_df["Sqft Area"],2)
#house_df=house_df.round({"Price/Sqft":2})
house_df.head()

,Property Types,Price,Street Addresses,City,State,Zipcode,Sqft Area,Beds,Baths,Links,Price/Sqft
0,House for Sale,279900.0,"147 Country Creek Ct,",Ballwin,MO,63011,2001.0,3,2.5,https://www.realtor.com//realestateandhomes-de...,139.88
1,House for Sale,350000.0,"349 Cherry Hill Dr,",Ellisville,MO,63011,2397.0,3,2.5,https://www.realtor.com//realestateandhomes-de...,146.02
2,House for Sale,340000.0,"402 Baberton Dr,",Ballwin,MO,63011,2419.0,4,2.5,https://www.realtor.com//realestateandhomes-de...,140.55
3,House for Sale,319900.0,"302 Wildbrier Dr,",Ballwin,MO,63011,2319.0,5,2,https://www.realtor.com//realestateandhomes-de...,137.95
4,House for Sale,199900.0,"607 Morewood Ct,",Manchester,MO,63011,1376.0,3,2,https://www.realtor.com//realestateandhomes-de...,145.28


In [92]:
house_data_df=house_df[["Property Types","Price/Sqft", "Street Addresses", "City", "Zipcode",]].copy()
house_data_df.head()

,Property Types,Price/Sqft,Street Addresses,City,Zipcode
0,House for Sale,139.88,"147 Country Creek Ct,",Ballwin,63011
1,House for Sale,146.02,"349 Cherry Hill Dr,",Ellisville,63011
2,House for Sale,140.55,"402 Baberton Dr,",Ballwin,63011
3,House for Sale,137.95,"302 Wildbrier Dr,",Ballwin,63011
4,House for Sale,145.28,"607 Morewood Ct,",Manchester,63011


In [93]:
house_data_df_renamed = house_data_df.rename(columns={'Property Types' : 'property_types', 'Zipcode': 'zipcode', \
                                                      'Street Addresses': 'street_address', 'City' : 'city'})

house_data_df_renamed.head()

,property_types,Price/Sqft,street_address,city,zipcode
0,House for Sale,139.88,"147 Country Creek Ct,",Ballwin,63011
1,House for Sale,146.02,"349 Cherry Hill Dr,",Ellisville,63011
2,House for Sale,140.55,"402 Baberton Dr,",Ballwin,63011
3,House for Sale,137.95,"302 Wildbrier Dr,",Ballwin,63011
4,House for Sale,145.28,"607 Morewood Ct,",Manchester,63011


In [94]:
schools_houses_merged_df = pd.merge(house_data_df_renamed, school_district_df_reduced, \
                                    on='zipcode', how='inner', suffixes=('_house', '_school'))

schools_houses_merged_df.head()

,property_types,Price/Sqft,street_address,city,zipcode,school_name,school_distrct,MO_raniking,national_ranking
0,House for Sale,139.88,"147 Country Creek Ct,",Ballwin,63011,Lafayette Sr. High,Rockwood R-Vi,1,431
1,House for Sale,139.88,"147 Country Creek Ct,",Ballwin,63011,West High,Parkway C-2,6,823
2,House for Sale,146.02,"349 Cherry Hill Dr,",Ellisville,63011,Lafayette Sr. High,Rockwood R-Vi,1,431
3,House for Sale,146.02,"349 Cherry Hill Dr,",Ellisville,63011,West High,Parkway C-2,6,823
4,House for Sale,140.55,"402 Baberton Dr,",Ballwin,63011,Lafayette Sr. High,Rockwood R-Vi,1,431


In [95]:
schools_houses_merged_df_clean = schools_houses_merged_df.dropna()

# 3. LOAD

## Now that our data is cleaned and ready for analysis, we want to make sure we store if for future use.  Due to the nature of our data retrieval through web scrapping, we decided to simply store the results into a Mongo DB for future use.  We summized that this would give us the most flexibility in storage and be an easy vehicle to allow us to easily store more scrapped data as we continue to collect.

In [96]:
# Import dependencies for the load
import pymongo
from pprint import pprint
import json

In [97]:
# Initialize PyMongo to work with MongoDB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [98]:
# Check if database exists already and drop it if so
dblist = client.list_database_names()
if "houses_schoools" in dblist:
    client.drop_database("houses_schoools")

In [99]:
# Create out DB and Collections
db = client.houses_schoools

house_collection = db.houses
schools_collection = db.schools_districts
house_schools_comparrison_collection = db.comparison_collection

In [100]:
# Create a list of dictionaries that hold the Mongo DB documents to be inserted
houses = house_df.to_dict(orient='records')
schools = school_df.to_dict(orient='records')
house_school_compare = schools_houses_merged_df.to_dict(orient='records')


print(f"Posting house records into collection of schools inside the 'houses_schoools' mongo database... \n")

print(f"Posting school/ district records into collection of hjouses inside the 'houses_schoools' mongo database...\n")

print(f"Posting school/ house comparrison records into collection of house/ schools comparrisonin inside the 'houses_schoools' mongo database...")



# Insert our houses and schools/ districts records indie teh MonDB DB for future analysis
house_collection.insert_many(houses)
schools_collection.insert_many(schools)
house_schools_comparrison_collection.insert_many(house_school_compare)

Posting house records into collection of schools inside the 'houses_schoools' mongo database... 

Posting school/ district records into collection of hjouses inside the 'houses_schoools' mongo database...

Posting school/ house comparrison records into collection of house/ schools comparrisonin inside the 'houses_schoools' mongo database...


In [101]:
# Check to make sure the house data was stored correctly
houses = house_collection.find()

for house in houses:
    print(house)

{'_id': ObjectId('5d2a1e3ef13f0e0594af3ebd'), 'Property Types': 'House for Sale', 'Price': 279900.0, 'Street Addresses': '147 Country Creek Ct,', 'City': 'Ballwin', 'State': 'MO', 'Zipcode': 63011, 'Sqft Area': 2001.0, 'Beds': '3', 'Baths': '2.5', 'Links': 'https://www.realtor.com//realestateandhomes-detail/147-Country-Creek-Ct_Ballwin_MO_63011_M84542-50248', 'Price/Sqft': 139.88}
{'_id': ObjectId('5d2a1e3ef13f0e0594af3ebe'), 'Property Types': 'House for Sale', 'Price': 350000.0, 'Street Addresses': '349 Cherry Hill Dr,', 'City': 'Ellisville', 'State': 'MO', 'Zipcode': 63011, 'Sqft Area': 2397.0, 'Beds': '3', 'Baths': '2.5', 'Links': 'https://www.realtor.com//realestateandhomes-detail/349-Cherry-Hill-Dr_Ellisville_MO_63011_M73739-32097', 'Price/Sqft': 146.02}
{'_id': ObjectId('5d2a1e3ef13f0e0594af3ebf'), 'Property Types': 'House for Sale', 'Price': 340000.0, 'Street Addresses': '402 Baberton Dr,', 'City': 'Ballwin', 'State': 'MO', 'Zipcode': 63011, 'Sqft Area': 2419.0, 'Beds': '4', 'Ba

In [102]:
# Check to make sure the school data was stored correctly
schools = schools_collection.find()

for school in schools:
    print(school)

{'_id': ObjectId('5d2a1e3ff13f0e0594af4394'), 'school_name': 'Lafayette Sr. High', 'school_distrct': 'Rockwood R-Vi', 'MO_raniking': '1', 'national_ranking': '431', 'city': 'Wildwood', 'state': 'MO', 'zipcode': 63011, 'full_address': '17050 Clayton Rd, Wildwood, Missouri 63011', 'phone_number': '(636) 733-4100', 'school_page_link_usn': 'https://www.usnews.com//education/best-high-schools/missouri/districts/rockwood-r-vi/lafayette-sr-high-11883'}
{'_id': ObjectId('5d2a1e3ff13f0e0594af4395'), 'school_name': 'Smithville High', 'school_distrct': 'Smithville R-II', 'MO_raniking': '2', 'national_ranking': '531', 'city': 'Smithville', 'state': 'MO', 'zipcode': 64089, 'full_address': '645 S Commercial Ave, Smithville, Missouri 64089', 'phone_number': '(816) 532-0405', 'school_page_link_usn': 'https://www.usnews.com//education/best-high-schools/missouri/districts/smithville-r-ii/smithville-high-11910'}
{'_id': ObjectId('5d2a1e3ff13f0e0594af4396'), 'school_name': 'Rockwood Summit Sr. High', 'sch

In [103]:
# Check to make sure the combined data was stored correctly
hs_compares = house_schools_comparrison_collection.find()

for record in hs_compares:
    print(record)

{'_id': ObjectId('5d2a1e3ff13f0e0594af44b7'), 'property_types': 'House for Sale', 'Price/Sqft': 139.88, 'street_address': '147 Country Creek Ct,', 'city': 'Ballwin', 'zipcode': 63011, 'school_name': 'Lafayette Sr. High', 'school_distrct': 'Rockwood R-Vi', 'MO_raniking': '1', 'national_ranking': '431'}
{'_id': ObjectId('5d2a1e3ff13f0e0594af44b8'), 'property_types': 'House for Sale', 'Price/Sqft': 139.88, 'street_address': '147 Country Creek Ct,', 'city': 'Ballwin', 'zipcode': 63011, 'school_name': 'West High', 'school_distrct': 'Parkway C-2', 'MO_raniking': '6', 'national_ranking': '823'}
{'_id': ObjectId('5d2a1e3ff13f0e0594af44b9'), 'property_types': 'House for Sale', 'Price/Sqft': 146.02, 'street_address': '349 Cherry Hill Dr,', 'city': 'Ellisville', 'zipcode': 63011, 'school_name': 'Lafayette Sr. High', 'school_distrct': 'Rockwood R-Vi', 'MO_raniking': '1', 'national_ranking': '431'}
{'_id': ObjectId('5d2a1e3ff13f0e0594af44ba'), 'property_types': 'House for Sale', 'Price/Sqft': 146.0